# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook we'll learn to use LASSO to select features by building on a pre-implemented LASSO solver. 

Contents:
* [Import GraphLab and Sales Data](#import)
* [Feature Engineering: Round 2](#feature)
* [Train Regression Model with L1 Penalty](#train)
  * [Q1: Chosen Features](#q1)
* [Choose the Best L1 Penalty](#choose)
  * [Q2: Best L1 Value](#q2)
  * [Q3: RSS on Test Data](#q3)
  * [Q4: Nonzero Weights](#q4)
* [Limit the Number of Nonzero Weights](#limit)
  * [Explore a Broad Range to Find Approximate L1 Penalties](#broad)
  * [Q5: Minimum L1 Penalty](#q5)
  * [Q6: Maximum L1 Penalty](#q6)
  * [Explore the Narrow Range to Minimize RSS](#narrow)
  * [Q7: L1 Penalty with Lowest RSS](#q7)
  * [Q8: Features with Nonzero Coefficients](#q8) 

<a id="import"></a>
## Import GraphLab and Sales Data

In [14]:
import graphlab as gl

In [15]:
sales = gl.SFrame('kc_house_data.gl/')

<a id="feature"></a>
## Feature Engineering: Round 2

As in week 2, let's make some new features for ourselves. We'll also need to import a couple functions from the math library. 

In [16]:
from math import log, sqrt

In [17]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'] * sales['bedrooms']

# In the dataset, the 'floors' column is defined as type string
# We have to convert it to a float before we can transform it
sales['floors'] = sales['floors'].astype(float)
sales['floors_square'] = sales['floors'] * sales['floors']

When we square a feature such as 'bedrooms' and 'floors', it increases the separation between cases with few instances (e.g. 1) and cases with many instances (e.g. 2), since 1 ^ 2 == 1 but 4 ^ 2 == 16. As such, the larger the instance the larger the effect will be. 

Conversely, taking the square root of a feature such as 'sqft_living' and 'sqft_lot', will decrease the separation between instances since doubling a feature shouldn't necessarily double the effect. 

<a id="train"></a>
## Train Regression Model with L1 Penalty

Let's train a model using all of the features from sales. To start, we'll select all of the features that we want to use.  

In [18]:
# Exclude the first 3 columns: id, date, and price
all_features = sales.column_names()[3:]

print('All Features: \n%s\n' % str(all_features))

All Features: 
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt', 'sqft_lot_sqrt', 'bedrooms_square', 'floors_square']



Now, we can train a model with a small L1 penalty and no L2 penalty.

In [19]:
small_l1_penalty = 1e10

all_features_model = gl.linear_regression.create(sales,
                                                 target='price',
                                                 features=all_features,
                                                 validation_set=None,
                                                 l1_penalty=small_l1_penalty,
                                                 l2_penalty=0.0,
                                                 verbose=False)

<a id="q1"></a>
### Quiz Question 1:
Which features have been chosen, i.e. which coefficients have not been reduced to 0?

In [20]:
all_features_model.get('coefficients').filter_by(0.0, 'value', exclude=True)

name,index,value,stderr
(intercept),None,270043.384256,None
bathrooms,None,6699.71721943,None
sqft_living,None,22.5213492398,None
grade,None,213.394377792,None
sqft_above,None,17.8152430447,None
sqft_living15,None,11.0941292701,None
sqft_living_sqrt,None,247.823348372,None


In [21]:
a = all_features_model.get('coefficients').filter_by(0.0, 'value', exclude=True)
n_rows = len(a)
a.print_rows(n_rows)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 270043.384256 |  None  |
|    bathrooms     |  None | 6699.71721943 |  None  |
|   sqft_living    |  None | 22.5213492398 |  None  |
|      grade       |  None | 213.394377792 |  None  |
|    sqft_above    |  None | 17.8152430447 |  None  |
|  sqft_living15   |  None | 11.0941292701 |  None  |
| sqft_living_sqrt |  None | 247.823348372 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]



<a id="choose"></a>
## Choose the Best L1 Penalty

The choose the best L1 penalty we'll explore multiple values using a validation set. To accomplish that, we'll do what we've done before and split sales into 3 datasets. 

In [22]:
(training_and_validation, testing) = sales.random_split(0.9, seed=1)
(training, validation) = sales.random_split(0.5, seed=1)

In [23]:
import numpy as np
from regression import get_residual_sum_of_squares

In [24]:
small_penalty_range = np.logspace(1, 7, num=13)
residuals_over_small_range = []

for p in small_penalty_range:
    model = gl.linear_regression.create(training,
                                        target='price',
                                        features=all_features,
                                        validation_set=None,
                                        l1_penalty=p,
                                        l2_penalty=0.0,
                                        verbose=False)
    
    rss = get_residual_sum_of_squares(model, validation, validation['price'])
    residuals_over_small_range.append(rss)

In [45]:
penalties_and_rss = gl.SFrame({'Penalties':small_penalty_range,
                               'RSS':residuals_over_small_range})

penalties_and_rss.head(4)

Penalties,RSS
10.0,3.96933187968e+14
31.6227766017,3.96933192192e+14
100.0,3.96933205551e+14
316.227766017,3.96933247795e+14


<a id="q2"></a>
### Quiz Question 2:
What is the best value for the `l1_penalty`, i.e. what `l1_penalty` has the lowest RSS?

In [46]:
min_rss_index = penalties_and_rss['RSS'].argmin()

penalty_with_smallest_rss = penalties_and_rss[min_rss_index]['Penalties']
min_rss = penalties_and_rss[min_rss_index]['RSS']

print('L1 Penalty: %0.2f\nLowest RSS: %0.2e\n' % (penalty_with_smallest_rss, min_rss))

L1 Penalty: 10.00
Lowest RSS: 3.97e+14



<a id="q3"></a>
### Quiz Question 3:
What is the RSS on Test data of the model with the best l1_penalty?

In [47]:
best_penalty = 10.00

model_w_best_penalty = gl.linear_regression.create(training,
                                                   target='price',
                                                   features=all_features,
                                                   validation_set=None,
                                                   l1_penalty=best_penalty,
                                                   l2_penalty=0.0,
                                                   verbose=False)

In [48]:
test_rss = get_residual_sum_of_squares(model_w_best_penalty, testing, testing['price'])

print('RSS on Test Data with best L1 Penalty: \n%0.2e\n' % test_rss)

RSS on Test Data with best L1 Penalty: 
8.24e+13



<a id="q4"></a>
### Quiz Question 4:
Using this value of L1 penalty, how many nonzero weights are there?

In [49]:
n_nonzero_coefficients = model_w_best_penalty['coefficients']['value'].nnz()

print("Number of Nonzero Coefficients: %i\n" % n_nonzero_coefficients)

Number of Nonzero Coefficients: 91



In [50]:
model_w_best_penalty['coefficients']

name,index,value,stderr
(intercept),None,16767.2865225,None
bedrooms,None,7098.52271832,None
bathrooms,None,20384.9007359,None
sqft_living,None,30.1640061731,None
sqft_lot,None,0.134267643378,None
floors,None,17343.1987875,None
waterfront,None,462926.524599,None
view,None,80178.8955573,None
condition,None,5389.32009259,None
grade,None,4769.46036056,None


<a id="limit"></a>
## Limit the Number of Nonzero Weights

Suppose we wanted a simple interpretable model with only a few features. How would we do that? Well that's what we're going to do next. 

We'll be trying to achieve 7 features (7 nonzeros), and to do that we'll implement a 2 phase procedure. 

1. Explore a broad range of L1 penalty values to find a narrow region of L1 penalty values where models are likely to have the desired number of features. 

2. Further explore that narrow region to find a specific L1 penalty that achieves the desired sparsity. As above, we will use a validation set to select the best L1 penalty value. 

<a id="broad"></a>
### Explore a Broad Range to Find Approximate L1 Penalties

First we'll define our range. 100 million to 10 billion. 

Then, we'll loop over that range to find a sub-range that gives us approximately the number of nonzeros that we're looking for. 

In [30]:
broad_penalty_range = np.logspace(8, 10, num=20)
broad_coefficients = []

for p in broad_penalty_range:
    model = gl.linear_regression.create(training,
                                        target='price',
                                        features=all_features,
                                        validation_set=None,
                                        l1_penalty=p,
                                        l2_penalty=0.0,
                                        verbose=False)
    
    n_nonzero_coefficients = model['coefficients']['value'].nnz()
    broad_coefficients.append(n_nonzero_coefficients)

In [31]:
penalties_and_coefficients = gl.SFrame({'Penalties':broad_penalty_range,
                                        'Coefficients':broad_coefficients})
# Change the order of the two columns
penalties_and_coefficients = penalties_and_coefficients[['Penalties', 'Coefficients']]

penalties_and_coefficients.tail(4)

Penalties,Coefficients
4832930238.57,8
6158482110.66,7
7847599703.51,1
10000000000.0,1


In [32]:
max_nonzeros = 7

# Breaking the SFrame into separate SArrays makes it
# a little nicer to work with while in the loop
penalties = penalties_and_coefficients['Penalties']
coefficients = penalties_and_coefficients['Coefficients']

# Loop backwards so coefficient values are ascending
for i in range(len(coefficients)-1, 0, -1):
    
    # Increment fewer_nonzeros and l1_penalty_min while the
    # coefficient value is less than max_nonzeros
    if coefficients[i] < max_nonzeros:
        fewer_nonzeros_than_max = coefficients[i]
        l1_penalty_min = penalties[i]
    
    # Find the first coefficient value greater than max_nonzeros
    # Set more_nonzeros and l1_penalty_max and finish with break
    if coefficients[i] > max_nonzeros:
        more_nonzeros_than_max = coefficients[i]
        l1_penalty_max = penalties[i]
        break

<a id="q5"></a>
### Quiz Question 5:
What is the value for `l1_penalty_min`?

In [38]:
print("L1 Penalty Min: %f\nNearest Greater Coefficient: %i\n" % (l1_penalty_max, more_nonzeros_than_max))

L1 Penalty Min: 4832930238.571753
Nearest Greater Coefficient: 8



<a id="q6"></a>
### Quiz Question 6:
What is the value for `l1_penalty_max`?

In [39]:
print("L1 Penalty Max: %e\nNearest Lower Coefficient: %i\n" % (l1_penalty_min, fewer_nonzeros_than_max))

L1 Penalty Max: 7.847600e+09
Nearest Lower Coefficient: 1



<a id="narrow"></a>
###Explore the Narrow Range to Minimize RSS

Finally, we'll train models over the range of `l1_penalty` values that we've found and select the one with the lowest RSS on our Validation dataset. 

In [40]:
narrow_residuals = []
narrow_nonzeroes = []

for p in np.linspace(l1_penalty_min, l1_penalty_max, 20):
    model = gl.linear_regression.create(training,
                                    target='price',
                                    features=all_features,
                                    validation_set=None,
                                    l1_penalty=p,
                                    l2_penalty=0.0,
                                    verbose=False)
    
    rss = get_residual_sum_of_squares(model, validation, validation['price'])
    narrow_residuals.append(rss)
    
    narrow_nonzeroes.append(model['coefficients']['value'].nnz())

In [41]:
narrow_l1_models = gl.SFrame({'Penalties':penalties,
                              'RSS':narrow_residuals,
                              'N Nonzeroes':narrow_nonzeroes})

narrow_l1_models = narrow_l1_models[['Penalties', 'RSS', 'N Nonzeroes']]
narrow_l1_models.head(4)

Penalties,RSS,N Nonzeroes
100000000.0,1.56022807889e+15,1
127427498.57,1.57391583727e+15,1
162377673.919,1.58880090582e+15,1
206913808.111,1.58659281991e+15,2


In [42]:
correct_sparsity_models = narrow_l1_models.filter_by(max_nonzeros, 'N Nonzeroes')

<a id="q7"></a>
###Quiz Question 7: 
What value of l1_penalty in our narrow range has the lowest RSS on the Validation dataset and has sparsity equal to max_nonzeros?

In [44]:
min_rss_index = correct_sparsity_models['RSS'].argmin()
min_rss_row = correct_sparsity_models[min_rss_index]

print('L1 Penalty: %f\nLowest RSS: %e\n' % (min_rss_row['Penalties'], min_rss_row['RSS']))


L1 Penalty: 7847599703.514623
Lowest RSS: 1.356067e+15



<a id="q8"></a>
### Quiz Question 8:
What features of the model with the lowest RSS have nonzero coefficients?

In [32]:
l1_penalty_w_7_coefficients = 7.8475997035e+09

model_w_7_coefficients = gl.linear_regression.create(sales,
                                                     target='price',
                                                     features=all_features,
                                                     validation_set=None,
                                                     l1_penalty=l1_penalty_w_7_coefficients,
                                                     l2_penalty=0.0,
                                                     verbose=False)

In [34]:
model_w_7_coefficients.get('coefficients').filter_by(0.0, 'value', exclude=True)

name,index,value,stderr
(intercept),None,160233.066821,None
bedrooms,None,3269.68167359,None
bathrooms,None,14435.3220651,None
sqft_living,None,24.7578164954,None
floors,None,5558.28404808,None
condition,None,171.124724447,None
grade,None,3135.80244358,None
sqft_above,None,23.5545092709,None
sqft_basement,None,0.73739569629,None
yr_built,None,0.160735023084,None
